## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Create a Deployment

- The Deployment definition is in the YAML file `manifests/deploy-example.yaml`

In [1]:
!kubectl apply -f manifests/deploy-example.yaml

deployment.apps/deploy-example created


## Let's look at the Deployment's YAML

**Note:**

- The number of `replicas` is set to 3.
- The Pod template's container is listening on `containerPort` 80.
- The Pod template's `labels` are `app: nginx` and `type: prod`
  - The Deployment's `matchLabels` match these labels
    - Therefore, the Deployment will create 3 replicas from the Pod template.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-example           # the deployment's name is deploy-example
spec:
  replicas: 3                    # the deployment defines three replicas
  revisionHistoryLimit: 3        # the deployment defines a maximum of three revisions in the history list
  selector:
    matchLabels:
      app: nginx                 # the deployment has two matchLabels that match the Pod template's labels below
      type: prod                 # app = nginx and type = prod
  template:
    metadata:
      name: myapp-pod
      labels:
        app: nginx               # the Pod template has two labels defined:
        type: prod               # app = nginx and type = prod
    spec:
      containers:
      - name: nginx-container    # the container's name is nginx-container
        image: nginx-container   # the container is based on the nginx:alpine image
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi    
        ports:
        - containerPort: 80      # the Pod template's container is listening on containerPort 80
```

In [2]:
#!type manifests/deploy-example.yaml # use this in Windows
!cat manifests/deploy-example.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-example
spec:
  replicas: 3
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: nginx
      env: prod
  template:
    metadata:
      labels:
        app: nginx
        env: prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi        
        ports:
        - containerPort: 80

## List Deployments

In [3]:
#!kubectl get deploy -o wide
!kubectl get deployments -o wide

NAME             READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES         SELECTOR
deploy-example   3/3     3            3           6s    nginx        nginx:alpine   app=nginx,env=prod


## Get detailed info. about the Deployment

- Notice `Replicas` tells us there are 3 desired, 3 updated, 3 total and 3 available Pod replicas.

In [4]:
#!kubectl describe deploy deploy-example
!kubectl describe deployment deploy-example

Name:                   deploy-example
Namespace:              default
CreationTimestamp:      Thu, 08 Feb 2024 03:57:47 +0100
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=nginx,env=prod
Replicas:               3 desired | 3 updated | 3 total | 3 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=nginx
           env=prod
  Containers:
   nginx:
    Image:      nginx:alpine
    Port:       80/TCP
    Host Port:  0/TCP
    Limits:
      cpu:     250m
      memory:  256Mi
    Requests:
      cpu:        100m
      memory:     128Mi
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSet

## List ReplicaSets

- Notice the name of the ReplicaSet consists of the Deployment name `deploy-example` and a randomly generated suffix.

In [5]:
#!kubectl get rs -o wide
!kubectl get replicasets -o wide

NAME                        DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES         SELECTOR
deploy-example-7b944585d7   3         3         3       11s   nginx        nginx:alpine   app=nginx,env=prod,pod-template-hash=7b944585d7


## Get detailed info. about all ReplicaSets

- Notice that we get detailed information about all ReplicaSets if we don't explicitly specify a replicaset.

In [6]:
#!kubectl describe rs
!kubectl describe replicasets

Name:           deploy-example-7b944585d7
Namespace:      default
Selector:       app=nginx,env=prod,pod-template-hash=7b944585d7
Labels:         app=nginx
                env=prod
                pod-template-hash=7b944585d7
Annotations:    deployment.kubernetes.io/desired-replicas: 3
                deployment.kubernetes.io/max-replicas: 4
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/deploy-example
Replicas:       3 current / 3 desired
Pods Status:    3 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=nginx
           env=prod
           pod-template-hash=7b944585d7
  Containers:
   nginx:
    Image:      nginx:alpine
    Port:       80/TCP
    Host Port:  0/TCP
    Limits:
      cpu:     250m
      memory:  256Mi
    Requests:
      cpu:        100m
      memory:     128Mi
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From                   Message
  ---

## List Pods

- Notice the name of the Pods consists of:
  - The Deployment name `deploy-example`
  - Followed by the ReplicaSet suffix
  - And finally a randomly generated suffix.

In [7]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                              READY   STATUS    RESTARTS   AGE   IP            NODE           NOMINATED NODE   READINESS GATES
deploy-example-7b944585d7-549hh   1/1     Running   0          16s   10.244.2.22   minikube-m03   <none>           <none>
deploy-example-7b944585d7-8t5cb   1/1     Running   0          16s   10.244.1.23   minikube-m02   <none>           <none>
deploy-example-7b944585d7-lbx6l   1/1     Running   0          16s   10.244.0.13   minikube       <none>           <none>


## Get detailed info. about the Pods

- Notice the name of the deployment is used here, so all three pods are described.

In [8]:
#!kubectl describe po deploy-example
!kubectl describe pod deploy-example

Name:             deploy-example-7b944585d7-549hh
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube-m03/192.168.49.4
Start Time:       Thu, 08 Feb 2024 03:57:47 +0100
Labels:           app=nginx
                  env=prod
                  pod-template-hash=7b944585d7
Annotations:      <none>
Status:           Running
IP:               10.244.2.22
IPs:
  IP:           10.244.2.22
Controlled By:  ReplicaSet/deploy-example-7b944585d7
Containers:
  nginx:
    Container ID:   docker://4cd6bd691a548f560865c503bfdc3d4f278302d9a921d4d3a90ce809fd961202
    Image:          nginx:alpine
    Image ID:       docker-pullable://nginx@sha256:f2802c2a9d09c7aa3ace27445dfc5656ff24355da28e7b958074a0111e3fc076
    Port:           80/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Thu, 08 Feb 2024 03:57:47 +0100
    Ready:          True
    Restart Count:  0
    Limits:
      cpu:     250m
      memory:  256Mi
    Requests:
    

## Delete the Deployment

In [9]:
!kubectl delete -f manifests/deploy-example.yaml

deployment.apps "deploy-example" deleted


## List Deployments, ReplicaSets and Pods

- Notice that deleting the Deployment also deleted the ReplicaSet and the Pods defined in the Deployment.

In [10]:
!kubectl get deploy
!kubectl get rs
!kubectl get po
#!kubectl get deployments
#!kubectl get replicasets
#!kubectl get pods

No resources found in default namespace.
No resources found in default namespace.
No resources found in default namespace.
